# RS-Client

In this demo we will see how to use the RS-Client Python library to call RS-Server.

## Configuration

In [ ]:
import getpass
import os

# Get the RS-Server website URL. 
# Note: this is None in local mode, we rather use the env vars RSPY_HOST_CADIP, RSPY_HOST_CATALOG, ...
url = os.getenv("RSPY_WEBSITE")

# Local mode or hybrid/cluster
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")

# In hybrid/cluster mode: read the API key
apikey = os.getenv("RSPY_APIKEY")
if (not local_mode) and (not apikey):
    apikey = getpass.getpass(f"Enter your API key from {url!r}:")

In [ ]:
# In local mode, install rs-client if missing
if local_mode:
    try:
        import rs_client
    except ModuleNotFoundError:
        !(cd $RSPY_WHL_DIR && pip install rs_client_libraries-*.whl)
        !opentelemetry-bootstrap -a install # install opentelemetry instrumentation for deps

## Initialisation

In [ ]:
from rs_client.rs_client import RsClient
from rs_common.config import ECadipStation, EPlatform

# Define the owner/user ID to read/write catalog collections.
# Your API key must give you the read/write permissions for this owner ID.
owner_id = "DemoUser"

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key
#   - Owner ID
#   - Logger (optional, a default one can be used)
generic_client = RsClient(url, apikey, owner_id, logger=None)

# From this generic instance, get an Auxip client instance
auxip_client = generic_client.get_auxip_client()

# Or get a Cadip client instance. Pass the cadip station and the platforms.
cadip_station = ECadipStation.CADIP
platforms = [EPlatform.S1A]
cadip_client = generic_client.get_cadip_client(cadip_station, platforms)

## Call single endpoints

In [ ]:
from datetime import datetime
import json
from time import sleep
from rs_common.config import EDownloadStatus

# Define some parameters
start_date = datetime(2014, 1, 1, 12, 0, 0)
stop_date = datetime(2024, 1, 1, 12, 0, 0)

# Timeout in seconds for the endpoints
TIMEOUT = 30

# Call the auxip and cadip endpoints
for client in [auxip_client, cadip_client]:

    # From these instances, we can e.g. search stations for products. 
    # We can limit to n products. The default sorting is by descending datetime ("-datetime")
    limit=5
    products = client.search_stations(start_date, stop_date, TIMEOUT, limit=limit)
    assert (len(products) == limit)
    print(f"First {client.station_name!r} product:\n{json.dumps(products[0], indent=2)}\n")

    filenames="\n".join([p["id"] for p in products])
    print(f"First {limit} {client.station_name!r} product names:\n{filenames}\n")

    # To sort e.g. by ascending datetimes:
    products = client.search_stations(start_date, stop_date, TIMEOUT, limit=5, sortby="+datetime")
    datetimes = "\n".join([p["properties"]["datetime"] for p in products])
    print(f"Ascending datetimes:\n{datetimes}\n")

    # We can stage e.g. the first product = download it from the station into the s3 object storage.
    # We can also download it locally into the server drive, but this is useful only in local mode.
    one_product_filename = products[0]["id"]
    client.staging(one_product_filename, TIMEOUT, s3_path=None, tmp_download_path=None)

    # Then we can check when the staging has finished
    while True:
        status = client.staging_status(one_product_filename, TIMEOUT)
        print (f"Staging status for {one_product_filename!r}: {status.value}")
        if status in [EDownloadStatus.DONE, EDownloadStatus.FAILED]:
            print("\n")
            break
        sleep(1)

In [ ]:
# Search cadip sessions by date interval
sessions = cadip_client.search_sessions(TIMEOUT, start_date=start_date, stop_date=stop_date)
print(f"First Cadip session:\n{json.dumps(sessions[0], indent=2)}\n")
ids=[s["id"] for s in sessions]
print_ids="\n".join(ids)
print(f"Session IDs for {platforms!r}:\n{print_ids}\n")

# Or get information for the cadip sessions e.g. #2 and #3
search_ids=ids[1:3]
search_sessions = cadip_client.search_sessions(TIMEOUT, session_ids=search_ids)
print(f"Cadip sessions information:\n{json.dumps(search_sessions, indent=2)}\n")